In [1]:
%matplotlib inline


# Sample pipeline for text feature extraction and evaluation


The dataset used in this example is the 20 newsgroups dataset which will be
automatically downloaded and then cached and reused for the document
classification example.

You can adjust the number of categories by giving their names to the dataset
loader or setting them to None to get the 20 of them.

Here is a sample output of a run on a quad-core machine::

  Loading 20 newsgroups dataset for categories:
  ['alt.atheism', 'talk.religion.misc']
  1427 documents
  2 categories

  Performing grid search...
  pipeline: ['vect', 'tfidf', 'clf']
  parameters:
  {'clf__alpha': (1.0000000000000001e-05, 9.9999999999999995e-07),
   'clf__n_iter': (10, 50, 80),
   'clf__penalty': ('l2', 'elasticnet'),
   'tfidf__use_idf': (True, False),
   'vect__max_n': (1, 2),
   'vect__max_df': (0.5, 0.75, 1.0),
   'vect__max_features': (None, 5000, 10000, 50000)}
  done in 1737.030s

  Best score: 0.940
  Best parameters set:
      clf__alpha: 9.9999999999999995e-07
      clf__n_iter: 50
      clf__penalty: 'elasticnet'
      tfidf__use_idf: True
      vect__max_n: 2
      vect__max_df: 0.75
      vect__max_features: 50000




In [52]:
from __future__ import print_function

from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

print(__doc__)

Automatically created module for IPython interactive environment


In [3]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [4]:
# #############################################################################
# Load some categories from the training set
categories = [
    'alt.atheism',
    'talk.religion.misc',
]
# Uncomment the following to do the analysis on all the categories
#categories = None

In [5]:
print("Loading 20 newsgroups dataset for categories:")
print(categories)

Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc']


In [6]:
data = fetch_20newsgroups(subset='train', categories=categories)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()

2018-09-23 03:59:31,638 INFO Downloading 20news dataset. This may take a few minutes.
2018-09-23 03:59:31,639 INFO Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


857 documents
2 categories



In [26]:
len(data.target)

857

In [31]:
import re
def clean(string):
    #return string
    clean_string = re.sub(r'\d+', 'number', string)
    clean_string = re.sub(r'Ä', 'Ae', clean_string)
    clean_string = re.sub(r'ä', 'ae', clean_string)
    clean_string = re.sub(r'Ö', 'Oe', clean_string)
    clean_string = re.sub(r'ö', 'oe', clean_string)
    clean_string = re.sub(r'Ü', 'Ue', clean_string)
    clean_string = re.sub(r'ü', 'ue', clean_string)
    clean_string = re.sub(r'ß', 'ss', clean_string)
    clean_string = re.sub(r'°', 'Grad', clean_string)
    clean_string = re.sub(r'[Zz][Bb]', 'zum Beispiel', clean_string)
    clean_string = re.sub(r'[Dd][Hh]', 'das heißt', clean_string)
    clean_string = re.sub(r'[Bb][Ss][Pp][Ww]', 'beispielsweise', clean_string)
    clean_string = re.sub(r'[Hh]allo', '', clean_string)
    clean_string = re.sub(r'[Hh]i', '', clean_string)
    clean_string = re.sub(r'[Hh]ey', '', clean_string)
    clean_string = re.sub(r'[Gg]uten\s[Mm]orgen', '', clean_string)
    clean_string = re.sub(r'[Gg]uten\s[Aa]bend', '', clean_string)
    
    clean_string = re.sub(r'(\([^)]*\))', ' ', clean_string)
    clean_string = re.sub(r'"', '', clean_string)
    clean_string = re.sub(r'\+', '', clean_string)
    clean_string = re.sub(r'-', '', clean_string)
    clean_string = re.sub(r',', '', clean_string)

    clean_string = re.sub(r'\'', '', clean_string)
    clean_string = re.sub(r'\.', '', clean_string)
    clean_string = re.sub(r'\s{2,}', ' ', clean_string)
    clean_string = re.sub(r'\s(?=\?)', ' ', clean_string)
    clean_string = re.sub(r'\?*(?=(?:\?))', '', clean_string)
    clean_string = clean_string.strip()
    #bitte, danke, und, eigentlich, überhaupt, git, wirklich
    return clean_string#.lower()

import Classification.config as cfg
import csv
data_set = []
for file in cfg.ALL_FILES:
    reader = csv.reader(open(file, 'r'), delimiter=';')
    for line in reader:
        try:
            data_set.append([clean(line[0]).lower(), line[1]])
        except Exception as e:
            print()

In [38]:
data_frame = pd.DataFrame(data_set)

['ist der ttstift lauter als die number generation?',
 'kann der tiptoi create auch hoerbuecher abspielen?',
 'ist da trotzdem auch die player funktion enthalten?',
 'geht die reichweite von der basis oder von der fernbedienung aus?',
 'wie lang piept das geraet?',
 'hat jemand erfahrung wie lange die batterien halten sowohl im sender als auch im empfaenger?',
 'wie dick ist der sender?',
 'wie sind denn die genauen masse des piepers in cm?',
 'sind die karten auf deutsch?',
 'kann mann die festplatte auch in einer playstation number pro einbauen?',
 'was passiert wenn die leer ist?',
 'kann ich die da formatieren?',
 'wie lange sind die batterien haltbar?',
 'hat jemand welche?',
 'ist die firma flower bud serioes?',
 'wie gross ist der durchmesser fuer die bepflanzung?',
 'wie kann das sein das ich post vom zoll kriege und weit fahren muss um ihn abzuholen?',
 'sind airpods dabei?',
 'gibt es garantie dazu?',
 'kann man das iphone number auch in oesterreich nutzen?',
 'ist der wsky g

In [53]:
# #############################################################################
# Define a pipeline combining a text feature extractor with a simple
# classifier
import xgboost as xgb

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC()),
    #('clf', xgb.XGBClassifier(max_depth=5, n_estimators=100, learning_rate=0.125, min_child_weight = 1)),
])

In [54]:
# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    #'clf__max_depth': (5, 6),
    #'clf__n_estimators': (100, 150),
    #'clf__learning_rate': (0.1, 0.125),
    #'clf__min_child_weight':(1, 2)
    #'clf__n_iter': (10, 50, 80),
}

In [ ]:
if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(data_frame[0], data_frame[1])
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))